# **텍스트데이터분석 프로젝트 전처리 코드**
#### **주제** : 게임사 5곳 특징, 전략, 사회적 가치 등 비교 분석 (넥슨, 크래프톤, 넷마블, 라이엇코리아, 엔씨소프트)
#### **1. Data Load & Data Merge**

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_colwidth', 200)

krafton_news = pd.read_csv('krafton_news.csv')
krafton_blog = pd.read_csv('krafton_blog.csv')
nexon_news = pd.read_csv('nexon_news.csv')
nexon_blog = pd.read_csv('nexon_blog.csv')
ncsoft_blog = pd.read_csv('ncsoft_blog.csv')
ncsoft_news = pd.read_csv('ncsoft_news.csv')
netmarble_blog = pd.read_csv('netmable_blog.csv')
netmarble_news = pd.read_csv('netmable_news.csv')
riot_blog = pd.read_csv('riot_blog.csv')
riot_news = pd.read_csv('riot_news.csv')

## 각 게임사별 라벨링 작업 진행
krafton_news['category'] = 'krafton_news'
krafton_blog['category'] = 'krafton_blog'
nexon_news['category'] = 'nexon_news'
nexon_blog['category'] = 'nexon_blog'
ncsoft_news['category'] = 'ncsoft_news'
ncsoft_blog['category'] = 'ncsoft_blog'
netmarble_news['category'] = 'netmarble_news'
netmarble_blog['category'] = 'netmarble_blog'
riot_news['category'] = 'riot_news'
riot_blog['category'] = 'riot_blog'

In [2]:
## 각 게임사별 데이터프레임 concat을 위해 slicing
krafton_news = krafton_news[['title', 'body', 'link', 'category']]
krafton_blog = krafton_blog[['title', 'body', 'link', 'category']]

nexon_news = nexon_news[['title', 'body', 'link', 'category']]
nexon_blog = nexon_blog[['title', 'body', 'link', 'category']]

ncsoft_news = ncsoft_news[['title', 'body', 'link', 'category']]
ncsoft_blog = ncsoft_blog[['title', 'body', 'link', 'category']]

netmarble_news = netmarble_news[['title', 'body', 'link', 'category']]
netmarble_blog = netmarble_blog[['title', 'body', 'link', 'category']]

riot_news = riot_news[['title', 'body', 'link', 'category']]
riot_blog = riot_blog[['title', 'body', 'link', 'category']]

df = pd.concat([krafton_news, krafton_blog, nexon_news, nexon_blog, ncsoft_news, ncsoft_blog, netmarble_news, netmarble_blog, riot_news, riot_blog], axis = 0)

#### **2. Data Preprocessing**
- 불용어 제거
- 결측치 제거
- 중복링크 제거

In [3]:
## 결측치 제거
df = df.loc[(df.title.notnull()) & (df.body.notnull())]

## 문서의 내용이 없는 경우 제거
df = df.loc[df.body != '사라진 페이지']

## title 불용어 제거
pattern1 = re.compile(r'<[^>]*>')
df.title = df.title.apply(lambda x: pattern1.sub('', x))  ## <b> 태그 제거
df.title.replace('↑', '상승', regex = True, inplace = True)  
df.title.replace('quot', ' ', regex = True, inplace = True)  ## 인용구 제거
df.title.replace('amp', ' ', regex = True, inplace = True)  ## & 마크 제거
df.title = df.title.str.replace('[^가-힣0-9a-zA-Z.,?!()\[\] ]', '', regex=True)  ## 한글, 숫자, 영어, 공백, 문장부호, 괄호까지만 남기고 제거
df.title = df.title.str.replace(r'http[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 http로 시작하는 링크가 있는 경우 제거
df.title = df.title.str.replace(r'www[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 www로 시작하는 링크가 있는 경우 제거
df.title = df.title.str.replace(r'blog[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 blog로 시작하는 링크가 있는 경우 제거
df.title = df.title.apply(lambda x: re.sub(r'\(\s*\)', ' ', x))  ## 텍스트 없이 괄호만 남은 경우에 괄호 제거

## body 불용어 제거
df.body.replace('\n', ' ', regex = True, inplace = True)  ## 줄바꿈 문자 제거
df.body.replace('↑', '상승', regex = True, inplace = True)
df.body.replace('↓', '하락', regex = True, inplace = True)
df.body = df.body.str.replace('[^가-힣0-9a-zA-Z.,?!%()\[\] ]', ' ', regex=True)  ## 한글, 숫자, 영어, 공백, 문장부호, 괄호까지만 남기고 제거
df.body = df.body.str.replace(r'http[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 http로 시작하는 링크가 있는 경우 제거
df.body = df.body.str.replace(r'www[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 www로 시작하는 링크가 있는 경우 제거
df.body = df.body.str.replace(r'blog[^[\(\sㄱ-ㅎㅏ-ㅣ가-힣]+', ' ', regex=True)  ## 제목에 blog로 시작하는 링크가 있는 경우 제거
df.body = df.body.apply(lambda x: re.sub(r'\(\s*\)', ' ', x))  ## 텍스트 없이 괄호만 남은 경우에 괄호 제거

## 게임사별 중복 링크 제거
## 첫 번째 값을 남기고 나머지를 제거했는데 먼저 나올수록 연관도가 높은 글이라고 생각해 이렇게 전처리했습니다
df = df.drop_duplicates(subset='link', keep='first')

## '라이엇 게임즈'랑 '라이엇게임즈'가 같은 의미라서 통합 작업
df['title'] = df.title.replace('라이엇게임즈', '라이엇 게임즈', regex = True)
df['body'] = df.body.replace('라이엇게임즈', '라이엇 게임즈', regex = True)

#### **3. Data Extraction**

In [4]:
df.to_csv('total_df.csv', index = False)